### Symbols in Market Watch

### Max Sharpe Weights

In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from dotenv import load_dotenv
import os

# -----------------------------
# Setup & MT5 initialization
# -----------------------------
load_dotenv()
login = int(os.getenv("AQUA_MT5_LOGIN"))
password = os.getenv("AQUA_MT5_PASSWORD")
server = os.getenv("AQUA_MT5_SERVER")

if not mt5.initialize(login=login, password=password, server=server):
    print("initialize() failed, error code =", mt5.last_error())
    quit()

# -----------------------------
# Config
# -----------------------------
risk_free_rate = 0.0  # annual risk-free rate

# FX pairs needed to convert index returns to USD
fx_map = {
    "GER40": "EURUSD",
    "JPN225": "USDJPY",
    "UK100": "GBPUSD",
    "AUS200": "AUDUSD"
}

# -----------------------------
# Helpers
# -----------------------------
def fetch_mt5_data(symbol, days=84):
    utc_to = pd.Timestamp.now()
    utc_from = utc_to - pd.Timedelta(days=days)

    rates = mt5.copy_rates_range(symbol, mt5.TIMEFRAME_D1, utc_from, utc_to)
    if rates is None:
        raise ValueError(f"Failed to fetch data for {symbol}")

    df = pd.DataFrame(rates)
    df["time"] = pd.to_datetime(df["time"], unit="s")
    df.set_index("time", inplace=True)
    return df

def calculate_log_returns(prices: pd.Series) -> pd.Series:
    return np.log(prices / prices.shift(1)).dropna()

def ewma_covariance(returns: pd.DataFrame, lambda_: float = 0.94) -> pd.DataFrame:
    """
    Compute EWMA covariance matrix (RiskMetrics style).
    returns: DataFrame of aligned returns (T × N)
    lambda_: decay factor (0.94 for daily)
    """
    r = returns.values
    T, N = r.shape

    # Initialize with sample covariance of first few observations
    init_window = min(5, T)
    cov = np.cov(r[:init_window].T)

    for t in range(init_window, T):
        x = r[t].reshape(-1, 1)
        cov = lambda_ * cov + (1.0 - lambda_) * (x @ x.T)

    return pd.DataFrame(cov, index=returns.columns, columns=returns.columns)

# -----------------------------
# Load factor signals
# -----------------------------
# ai_factor_signal_aqua_funded.csv:  asset \t factor_signal
expected_returns_df = pd.read_csv("ai_factor_signal_aqua.csv", sep=",", header=None)
expected_returns_df.columns = ["asset", "factor_signal"]

# -----------------------------
# Z-score the factor signals
# -----------------------------
mean_signal = expected_returns_df["factor_signal"].mean()
std_signal = expected_returns_df["factor_signal"].std()

expected_returns_df["zscore_signal"] = (
    (expected_returns_df["factor_signal"] - mean_signal) / std_signal
)

# Convert to dict
expected_returns_dict = dict(
    zip(expected_returns_df["asset"], expected_returns_df["zscore_signal"])
)

symbols = list(expected_returns_dict.keys())

# -----------------------------
# Fetch price data for assets
# -----------------------------
data = {symbol: fetch_mt5_data(symbol) for symbol in symbols}

# -----------------------------
# Fetch FX data and compute FX log returns
# -----------------------------
fx_symbols = set(fx_map[s] for s in symbols if s in fx_map)
fx_data = {fx: fetch_mt5_data(fx) for fx in fx_symbols}


fx_returns = {}
for fx, df_fx in fx_data.items():
    lr = calculate_log_returns(df_fx["close"])

    # If pair is quoted as USDXXX (e.g., USDJPY, USDHKD), flip sign to get XXXUSD log return
    if fx.startswith("USD"):
        lr = -lr  # log(1/x) = -log(x)

    fx_returns[fx] = lr

# -----------------------------
# Compute USD-adjusted log returns for all assets
# -----------------------------
asset_returns = {}

for symbol in symbols:
    idx_lr = calculate_log_returns(data[symbol]["close"])

    if symbol in fx_map:
        fx_symbol = fx_map[symbol]
        fx_lr = fx_returns[fx_symbol]

        # Align index and FX by date
        combined = pd.concat([idx_lr, fx_lr], axis=1, join="inner")
        combined.columns = ["idx", "fx"]

        # USD-adjusted return: r_usd = r_index_local + r_fx
        asset_returns[symbol] = combined["idx"] + combined["fx"]
    else:
        # Already in USD terms
        asset_returns[symbol] = idx_lr

# -----------------------------
# Build aligned returns matrix
# -----------------------------

returns = pd.DataFrame(asset_returns)[symbols].dropna(how="any")

# -----------------------------
# Compute daily volatilities
# -----------------------------
daily_vols = returns.std().values  # vector aligned with symbols

# -----------------------------
# EWMA Covariance matrix
# -----------------------------
lambda_ = 0.94
cov_matrix = ewma_covariance(returns, lambda_=lambda_)
cov_matrix = cov_matrix + 1e-6 * np.eye(len(cov_matrix))

# -----------------------------
# Expected daily returns vector (volatility-scaled)
# -----------------------------
IC_estimate = 0.05
zscores = np.array([expected_returns_dict[symbol] for symbol in symbols])

if IC_estimate < 0:
    zscores = -zscores
    IC_estimate = abs(IC_estimate)

# Multiply by volatility
expected_returns = IC_estimate * zscores * daily_vols

# -----------------------------
# Max Sharpe optimization
# -----------------------------
def max_sharpe_ratio(expected_returns, cov_matrix, risk_free_rate=0.0):
    num_assets = len(expected_returns)

    def neg_sharpe(weights):
        port_ret = np.dot(weights, expected_returns)
        port_vol = np.sqrt(weights.T @ cov_matrix @ weights)
        if port_vol <= 0:
            return 1e6
        return -(port_ret - risk_free_rate) / port_vol

    # REMOVE dollar-neutral constraint
    constraints = [
        {"type": "ineq", "fun": lambda w: 1.0 - np.sum(w**2)}  # L2 leverage only
    ]

    # Allow long/short
    bounds = [(-1, 1)] * num_assets

    # Initial guess: simple equal-weight (no neutrality enforcement)
    init_guess = np.ones(num_assets) / num_assets

    result = minimize(
        neg_sharpe,
        init_guess,
        bounds=bounds,
        constraints=constraints,
        method="SLSQP",
    )

    if not result.success:
        print("Warning: optimization did not fully converge:", result.message)

    weights = result.x
    daily_ret = np.dot(weights, expected_returns)
    daily_vol = np.sqrt(weights.T @ cov_matrix @ weights)

    return weights, daily_ret, daily_vol


# -----------------------------
# Run optimization
# -----------------------------
weights, daily_port_return, daily_port_vol = max_sharpe_ratio(
    expected_returns, cov_matrix, risk_free_rate=risk_free_rate
)

# -----------------------------
# Export optimal weights
# -----------------------------
weights_df = pd.DataFrame({
    "asset": symbols,
    "weight": weights
})
weights_df.to_csv("aqua_funded_optimal_portfolio_weights.csv", header=False, index=False)
print("\nWeights exported to aqua_funded_optimal_portfolio_weights.csv")

# -----------------------------
# Annualized statistics
# -----------------------------
annual_port_return = daily_port_return * 252
annual_port_vol = daily_port_vol * np.sqrt(252)
annual_sharpe = (annual_port_return - risk_free_rate) / annual_port_vol

# -----------------------------
# Output
# -----------------------------

print("\n--- MODEL Portfolio Statistics ---")
print(f"Daily Return: {daily_port_return:.6%}")
print(f"Annualized Return: {annual_port_return:0.2%}")
print(f"Daily Volatility: {daily_port_vol:.6%}")
print(f"Annualized Volatility: {annual_port_vol:.2%}")
print(f"Annualized Sharpe Ratio: {annual_sharpe:.2f}")

# -----------------------------
# Shutdown MT5
# -----------------------------
mt5.shutdown()

: 

In [ ]:
fx_returns["USDJPY"]

In [ ]:
returns[["USDJPY", "JPN225"]].tail()

### Volatility Targeted Weights using EWMA estimates

In [ ]:
import pandas as pd
import numpy as np

# Align weights to returns.columns order
weights_df = weights_df.set_index("asset").reindex(returns.columns)

# Extract weight vector
weights = weights_df["weight"].values
print("Loaded weights (aligned):")
print(weights)
print("\nAsset order:")
print(list(returns.columns))

# ============================================================
# --- EWMA COVARIANCE MATRIX (DAILY) ---
# ============================================================
lambda_ = 0.94  # RiskMetrics default
rets = returns.values
n = rets.shape[1]

# Start with sample covariance as initial value
ewma_cov = returns.cov().values.copy()

# EWMA update: use r_t at time t
for t in range(1, len(rets)):
    r = rets[t].reshape(-1, 1)
    ewma_cov = lambda_ * ewma_cov + (1 - lambda_) * (r @ r.T)

# Portfolio EWMA daily volatility
daily_portfolio_returns = returns @ weights
ewma_daily_vol = np.sqrt(weights.T @ ewma_cov @ weights)

# ============================================================
# --- BASE PORTFOLIO STATISTICS ---
# ============================================================

ewma_annual_vol = ewma_daily_vol * np.sqrt(252)
ewma_annual_return = daily_portfolio_returns.mean() * 252

# Sharpe ratio (realized, ex-post)
ewma_sharpe = ewma_annual_return / ewma_annual_vol

print("\n--- EWMA VOLATILITY FORECAST ---")
print(f"Daily Volatility: {ewma_daily_vol:0.2%}")
print(f"Annual Volatility: {ewma_annual_vol:0.2%}")
print(f"Daily Portfolio Return (realized): {daily_portfolio_returns.mean():0.2%}")
print(f"Annual Portfolio Return (realized): {ewma_annual_return:0.2%}")
print(f"Sharpe Ratio (realized): {ewma_sharpe:0.4f}")

# ============================================================
# --- SCALE PORTFOLIO TO TARGET VOLATILITY USING EWMA ---
# ============================================================

target_annual_vol = 0.05  # e.g. 10% annual target

# Convert target annual vol to daily
target_daily_vol = target_annual_vol / np.sqrt(252)

# Scale factor in DAILY units
scale_factor = target_daily_vol / ewma_daily_vol
scaled_weights = weights * scale_factor

# Recompute volatility after scaling
scaled_daily_vol = np.sqrt(scaled_weights.T @ ewma_cov @ scaled_weights)
scaled_annual_vol = scaled_daily_vol * np.sqrt(252)

# Compute scaled returns
scaled_daily_returns = returns @ scaled_weights
scaled_avg_daily_return = scaled_daily_returns.mean()
scaled_annual_return = scaled_avg_daily_return * 252

# Sharpe ratio after scaling (realized)
scaled_sharpe = scaled_annual_return / scaled_annual_vol

print(f"\n--- SCALED PORTFOLIO (Target Vol = {target_annual_vol:.0%}) ---")
print(f"Factor: {scale_factor:.4f}")
print(f"Daily Volatility (EWMA): {scaled_daily_vol:0.2%}")
print(f"Annual Volatility (EWMA): {scaled_annual_vol:0.2%}")


print(f"\n--- EX-ANTE ---")
print(f"Daily Portfolio Return: {scaled_avg_daily_return:0.2%}")
print(f"Annual Portfolio Return: {scaled_annual_return:0.2%}")
print(f"Sharpe Ratio: {scaled_sharpe:0.4f}")


# --- EXPORT SCALED WEIGHTS TO CSV ---
scaled_df = pd.DataFrame({
    "asset": returns.columns,
    "scaled_weight": scaled_weights
})

scaled_df.to_csv("aqua_funded_scaled_weights.csv", index=False)
print("\nScaled weights exported to aqua_funded_scaled_weights.csv")

### Lot Sizing

In [ ]:
import math
import pandas as pd
import MetaTrader5 as mt5
import numpy as np

# ============================
# USER CONFIG
# ============================
EQUITY = 200000.90  # set your account equity

# FX-exempt symbols (no price in formula)
FX_EXEMPT = ["USDJPY", "USDCHF", "USDCAD"]

# Global index → FX mapping
INDEX_FX_MAP = {
    "GER40": "EURUSD",
    "JPN225": "USDJPY",
    "UK100": "GBPUSD",
    "AUS200": "AUDUSD"
}

# ============================
# MT5 INITIALIZATION
# ============================
mt5.initialize()

def get_latest_price(symbol):
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        return None
    return tick.bid

def fetch_prices(assets):
    return {a: get_latest_price(a) for a in assets}

def fetch_index_fx_rates():
    return {idx: get_latest_price(fx) for idx, fx in INDEX_FX_MAP.items()}

# ============================
# LOAD CSV FILES
# ============================
contract_df = pd.read_csv("aqua_funded_contract_size.csv")

# Normalize column names
contract_df.columns = contract_df.columns.str.strip().str.lower()


# Merge contract sizes + weights
df = contract_df.merge(scaled_df, on="asset", how="left")

# ============================
# FETCH PRICES FROM MT5
# ============================
all_assets = df["asset"].tolist()
latest_prices = fetch_prices(all_assets)
index_fx_rates = fetch_index_fx_rates()

df["latest_price"] = df["asset"].map(latest_prices)

# ============================
# FETCH NET POSITIONS FROM MT5
# ============================

def fetch_net_positions():
    positions = mt5.positions_get()
    if positions is None:
        return {}

    net = {}

    for p in positions:
        symbol = p.symbol
        volume = p.volume if p.type == 0 else -p.volume  # BUY = +, SELL = -
        net[symbol] = net.get(symbol, 0) + volume

    return net

net_positions = fetch_net_positions()
df["current_holdings"] = pd.to_numeric(df["asset"].map(net_positions).fillna(0), errors="coerce")

# ============================
# LOT SIZE CALCULATION
# ============================
def compute_lot(row):
    asset = row["asset"]
    weight = row["scaled_weight"]
    contract_size = row["contract_size"]
    price = row["latest_price"]

    if pd.isna(weight) or pd.isna(contract_size):
        return None
    if contract_size == 0:
        return None

    # FX-exempt assets
    if asset in FX_EXEMPT:
        return (weight * EQUITY) / contract_size

    if price is None or price == 0:
        return None

    # Global index → convert to USD
    if asset in INDEX_FX_MAP:
        fx_rate = index_fx_rates.get(asset)
        if fx_rate is None or fx_rate == 0:
            return None
        if asset == "JPN225":
            fx_rate = 1 / fx_rate
        price = price * fx_rate
        if price == 0:
            return None

    denominator = price * contract_size
    if denominator == 0:
        return None

    return (weight * EQUITY) / denominator


# ============================
# CURRENT WEIGHT CALCULATION
# ============================
def compute_current_weight(row):
    asset = row["asset"]
    lot = row.get("current_holdings", 0)
    contract_size = row["contract_size"]
    price = row["latest_price"]

    if pd.isna(lot) or pd.isna(contract_size):
        return None
    if contract_size == 0:
        return None

    # FX-exempt assets
    if asset in FX_EXEMPT:
        position_value = lot * contract_size
        return position_value / EQUITY

    if price is None or price == 0:
        return None

    # Global index → convert to USD
    if asset in INDEX_FX_MAP:
        fx_rate = index_fx_rates.get(asset)
        if fx_rate is None or fx_rate == 0:
            return None
        if asset == "JPN225":
            fx_rate = 1 / fx_rate
        price = price * fx_rate
        if price == 0:
            return None

    position_value = lot * price * contract_size
    return position_value / EQUITY


from decimal import Decimal, ROUND_HALF_UP
import math

def safe_quantize(x, places="0.0001"):
    if x is None or (isinstance(x, float) and math.isnan(x)):
        return None
    return Decimal(str(x)).quantize(Decimal(places), rounding=ROUND_HALF_UP)

# Compute current weights
df["current_weight"] = df.apply(compute_current_weight, axis=1)
df["current_weight"] = df["current_weight"].apply(lambda x: safe_quantize(x, "0.0001"))

# Compute target lot sizes
df["target_lot_size"] = df.apply(compute_lot, axis=1)
df["target_lot_size"] = df["target_lot_size"].apply(lambda x: safe_quantize(x, "0.01"))

# ============================

df["target_lot_size"] = pd.to_numeric(df["target_lot_size"], errors="coerce")

df["difference"] = df["target_lot_size"] - df["current_holdings"]
df["difference"] = pd.to_numeric(df["difference"], errors="coerce").round(2)
df.rename(columns={"scaled_weight": "target_weight"}, inplace=True)
# ============================
# OUTPUT
# ============================
print(df[["asset", "latest_price", "current_holdings", "target_lot_size", "difference"]].dropna())
df.dropna().to_csv("aqua_funded_lot_sizes_output.csv", index=False)

# ============================
# GROSS TOTAL LOT SIZE
# ============================
df["abs_target_lot_size"] = df["target_lot_size"].abs()
df["abs_current_holdings"] = df["current_holdings"].abs()
gross_target_lot_size = df["abs_target_lot_size"].sum()
gross_current_holdings = df["abs_current_holdings"].sum()

print(f"\nGross Total Target Lot Size (absolute): {gross_target_lot_size:.2f}")
print(f"Gross Current Holdings (absolute): {gross_current_holdings:.2f}")

mt5.shutdown()

In [ ]:
df

In [ ]:
# --- 1. Cross‑sectional dispersion across assets ---

# daily dispersion (std across assets for each day)
dispersion = returns.std(axis=1)

# convert to percentage strings
dispersion_pct = dispersion.apply(lambda x: f"{x:.2%}")
print("\n--- DAILY CROSS-SECTIONAL DISPERSION ---")
print(dispersion_pct.tail(10))


# --- 2. Align weights with returns ---

# convert weights df into a Series indexed by asset
current_weights = df.set_index('asset')['current_weight']

# align weights to returns columns (critical step)
current_weights = current_weights.reindex(returns.columns).astype(float)

# optional: check for mismatches
missing_in_weights = set(returns.columns) - set(current_weights.index)
missing_in_returns = set(current_weights.index) - set(returns.columns)
if missing_in_weights:
    print("Assets in returns but missing weights:", missing_in_weights)
if missing_in_returns:
    print("Assets in weights but missing returns:", missing_in_returns)


# --- 3. Compute weighted portfolio returns (ex‑post) ---

# multiply each column by its weight and sum across assets
today_port_return = (returns.tail(1) * current_weights)
real_time_vol = today_port_return.std(axis=1) * np.sqrt(252)



print(f"\ntoday's portfolio volatility (REAL-TIME): ")

print(f"{real_time_vol.iloc[0]:.2%}")

In [ ]:
today_port_return

In [ ]:
returns.tail(1) #* current_weights

In [ ]:
# basic descriptive statistics
desc_stats = dispersion.describe()

# add skewness and kurtosis
desc_stats.loc["skewness"] = dispersion.skew()
desc_stats.loc["kurtosis"] = dispersion.kurtosis()

print(desc_stats)

